# Preprocessing the Datasets for Classification

In [1]:
# imports
import pandas as pd

## The Good vs Bad Plans Dataset
Stored in an Excel file titled "VSADataForClassification.xlsx", consisting of 4 sheets.

### Sheet names:
1. Export Worksheet - All the data from the next two sheets combined
2. GoodPlans - All good plans
3. BadPlans - All bad plans
4. SQL - Contains SQL code for retrieving from the database

### Columns:
1. PLANID - Plan ID Number
2. ActualLengthOfPlan - Number of quarters total
3. NumberOfQtrWithoutCourses - Number of quarters with no courses
4. NumberOfQtrWithCourses - Number of quarters with at least one course
5. TotalNumberOfCourses - Total courses in the plan
6. AvgNumberOfCoursesPerQuarter - Average number of courses in a quarter
7. TotalNumberOfYears - Number of years it takes to complete the plan
8. NumberofYearswithCOurse - Number of years it takes to complete all the courses, not including breaks
9. GapDelta
10. FinalScore - Binary value of the plan, where 0 is Bad, and 1 is Good

In [2]:
# opening and storing the files of the dataset
xl = pd.ExcelFile('VSADataForClassification.xlsx')
df_good = xl.parse('GoodPlans')
df_bad = xl.parse('BadPlans')

### Balancing the Dataset
One of the important things in Machine Learning is having a balanced dataset, where a certain class is much more represented in the data than another. One method of balancing the dataset is to duplicate the minority data so that there is an equal number of data for each class.

There are 286 good plans, and 1225 bad plans, meaning that the data is immbalanced. Therefore, to make it balanced, the good data is to be duplicated.

In [3]:
# for generating random indices
import random

# randomly duplicate the good plans
# so that there are an equal number of bad & good plans
df_dup = df_bad.copy()
for i in range(df_good.shape[0]):
    df_dup.loc[i] = df_good.loc[i]    
for i in range(286,df_bad.shape[0]):
    index = random.randint(0,df_good.shape[0] - 1)
    df_dup.loc[i] = df_good.loc[index]
    
# shuffle the duplicated good plans
df_dup = df_dup.copy()
df_dup = df_dup.sample(frac=1).reset_index(drop=True)

In [4]:
# create a combined dataset of good and bad plans
df_total = pd.concat([df_dup, df_bad])
# randomize the rows
df_total = df_total.sample(frac=1).reset_index(drop=True)
# drop uneeded features
df_total = df_total.drop(['PLANID'], axis=1)

In [6]:
# export the datasets
df_dup.to_csv('VSA_new_good.csv', index=False)
df_bad.to_csv('VSA_new_bad.csv', index=False)

# export labels
labels = df_total[['FinalScore']].copy()
labels.to_csv('VSA_labels.csv')
# export dataset
df_total = df_total.drop(['FinalScore'], axis=1)
df_total.to_csv('VSA_new_total.csv', index=False)

### Normalizing the Dataset
In a dataset, different features may have different scales. For example, if one feature is the age of a person, then the scale could be anywhere from 0-70. Meanwhile, another feature could be the length of their thumb, which could range anywhere from 2-10 cm. Because of the difference in scale, the effect of one feature on a classifier may have different weight.

Therefore, it is important to scale the features and normalize them so that they have a similar scale. The features I am using are the averages, meaning that the first four columns will not be used.

The following expression will be used to calculate the mean centered value:

__(feature_mean - x) / feature_std__

In [7]:
# mean center the dataset except for the labels column
df_norm = df_total.copy()
for column in df_norm:
    std = df_total[column].std()
    mean = df_total[column].mean()
    for i in range(df_total.shape[0]):
        x = df_total.loc[i, column]     
        df_norm.loc[i, column] = (mean - x) / std
            
# export this normalized dataset
df_norm.to_csv('VSA_new_norm.csv', index=False)

# Sequential Data
Stored in a file called 'studyplans.csv'. 

### Columns:
1. ID - The ID of the row
2. PlanID - The ID of the plan
3. QuarterID - The ID of the quarter. 4 is summer, 1 is fall, 2 is winter, and 3 is spring
4. YearID - The year of the quarter
5. CourseID - The class course number in the database

In [8]:
# read the file and load the data
df = pd.read_csv('studyplans.csv')

### Mapping each course to a plan ID in order
Each course is mapped to a plan, in the sequential order which the student will take them. Each plan comprises one 'sentence' in which the courses in order are the words, and ends with the word "EOS".

Example: Plan 9 consists of the courses 921 1170 264 1133. Therefore its corresponding sentence is: "921 1170 264 1133 EOS"

In [9]:
# dictionary maps PlanID to a sequential list of sources
plan_dict = {}
for i, row in df.iterrows():
    if row['PlanID'] not in plan_dict.keys():
        course_list = []
        plan_dict[row['PlanID']] = course_list
    plan_dict[row['PlanID']].append(row['CourseID'])
    
# convert to a dataframe
df_map = df[['PlanID']].copy()
df_map['Sentence'] = 0
df_map = df_map.drop_duplicates(['PlanID'])
df_map['ind'] = df_map['PlanID']
df_map = df_map.set_index('ind')
for key in plan_dict:
    sentence = ""
    for course in plan_dict[key]:
        sentence += str(course) + " "
    sentence += "EOS"
    df_map.loc[key, 'Sentence'] = sentence

In [10]:
# create CSV
df_map.to_csv('studyplan_sentences.csv')

# make a document containing all sentences
document = ""
for key in plan_dict:
    document += df_map.loc[key, 'Sentence'] + " "
with open('studyplan_document.txt', 'w') as textfile:
    textfile.write(document)